# **PROGETTO FOUNDATION OF COMPUTER SCIENCE**

## Fogliata Nadia 820908, Varcasia Davide 784328

## Import dei dati e librerie utilizzate

In [1]:
import pandas as pd
from itertools import chain
import numpy as np
import re
import string

In [79]:
book = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz', delimiter = ';', encoding = 'latin-1', compression = 'gzip')
book_rat = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz', delimiter = ';', encoding = 'latin-1', compression = 'gzip')
user = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz', delimiter = ';', encoding = 'latin-1', compression = 'gzip')
good_book = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz', delimiter = ',', encoding = 'latin-1', compression = 'gzip')
good_book_rat = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz', delimiter = ',', encoding = 'latin-1', compression = 'gzip')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## RICHIESTA 1: Normalize the location field of Users dataset, splitting into city, region, country

In [3]:
user.head(3)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN


In [ ]:
print('Dimensioni dataset user: ',user.shape)
print('Numero di utenti: ', len(pd.unique(user['User-ID'])))
print('Numero di location univoche: ', len(pd.unique(user['Location'])))

Dimensioni dataset user:  (278858, 3)
Numero di utenti:  278858
Numero di location univoche:  57339


In [87]:
split = user['Location'].str.split(',', expand=True)
split.head(3)

,0,1,2,3,4,5,6,7,8
0,nyc,new york,usa,None,None,None,None,None,None
1,stockton,california,usa,None,None,None,None,None,None
2,moscow,yukon territory,russia,None,None,None,None,None,None


Se tutte le righe rispettassero lo schema dovrebbero esserci solo le prime 3 colonne 

In [6]:
indici = list()
for i in range(3,9):
    indici.append(split.loc[~split[i].isnull(),].index)
indici = list(chain.from_iterable(indici))
indici = np.unique(indici)
print('In totale ci sono: ', len(indici), ' osservazioni che non rispettano lo schema city, region, country')

In totale ci sono:  1508  osservazioni che non rispettano lo schema city, region, country


Notiamo che non tutti i campi rispettano il formato: 'city', 'region', 'country' allora puliamo il testo affinché la situazione migliori:

In [9]:
def clean_text(text):
    text = text.lower()
    text = re.sub('https?://\S+|www\.\S+', '', text) #Url remove
    text = re.sub(',,', ',', text)
    text = re.sub('[%s]' % re.escape('!"\\#\\$%\\&\'\\(\\)\\*\\+\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~'), '', text) #Remove punctuation
    text = re.sub('\n', '', text) #Remove \n
    text = re.sub('\d+', '', text) #Number not string
    text = re.sub(' +', ' ', text) #Remove space > 1
    text = text.split(',')
    text = [x.strip(' ') for x in text]
    text = [x for x in text if x != '']
    text = [x for x in text if x != 'na']
    text = ', '.join(text)
    return text

Aggiungiamo la colonna con il testo di Location pulito:

In [10]:
user['clean_location'] = user['Location'].apply(clean_text)

In [18]:
user.loc[144:148,:]

,User-ID,Location,Age,clean_location
144,145,"inverness, scotland, united kingdom",NaN,"inverness, scotland, united kingdom"
145,146,"la, california, usa",NaN,"la, california, usa"
146,147,"west haverstraw, new york, usa",NaN,"west haverstraw, new york, usa"
147,148,"essex, england, united kingdom",63.0,"essex, england, united kingdom"
148,149,"ljubljana, n/a, slovenia",NaN,"ljubljana, slovenia"


I campi di location possono avere:
- Una sola informazione
- Due informazioni
- Tre informazioni (vanno bene)
- Più di tre informazioni

In [19]:
one, two, three = [],[], []
for i,row in user.iterrows():
    if len(user.loc[i,'clean_location'].split(',')) == 1:
        one.append(i)
    if len(user.loc[i,'clean_location'].split(',')) ==2:
        two.append(i)
    if len(user.loc[i,'clean_location'].split(',')) > 3:
        three.append(i)

In [20]:
print('Location con 1 dettaglio geografico: ', len(one))
print('Location con 2 dettagli geografici: ', len(two))
print('Location con + di 3 dettagli geografici: ', len(three))

Location con 1 dettaglio geografico:  3420
Location con 2 dettagli geografici:  15661
Location con + di 3 dettagli geografici:  1113


Lista di campi che non rispettano lo schema 'city', 'region', 'country':

In [21]:
incomplete = list(chain.from_iterable([one, two, three]))

In [22]:
user[['city', 'region', 'country']] = user['clean_location'].str.split(',', expand=True)[[0,1,2]]
user.loc[incomplete, ['city', 'region', 'country']] = ''

Salviamo nel dataset respect i campi che rispettano lo schema, e salviamo i valori univoci di 'city', 'region', 'country'

In [23]:
respect = user
respect = respect.drop(incomplete)
respect = respect.drop_duplicates(['clean_location'])
city, region, country = pd.unique(respect['city']),  pd.unique(respect['region']),  pd.unique(respect['country'])
clean_location = respect['clean_location']

Gestiamo i casi che non rispettano lo schema nel seguente modo:
- Una sola informazione: viene replicata 3 volte
- Due informazioni: verifichiamo che la stringa  sia presente in respect "clean_location" e se c'è un solo match, assegnamo i tre valori, altrimenti replichiamo due volte il primo dei due valori
- Più di tre informazioni: verifichiamo che una delle stringhe che rispetta lo schema sia contenuta nel dataset incompleti che contiene solo i casi in cui ci sono più di 3 informazioni; se è contenuto gli associamo i rispettivi valori, altrimenti teniamo solo gli ultimi 3

In [24]:
for i in one:
    user.loc[i,['city', 'region', 'country']] = [user.loc[i,'clean_location']]*3

In [25]:
for i in two:
    not_know = user.loc[i,'clean_location']
    not_know_li = not_know.split(',')
    match = respect[respect['clean_location'].str.contains(not_know, na = False)]
    if match.shape[0] == 1:
        user.loc[i, ['city', 'region', 'country']] = match[['city', 'region', 'country']].values
    else: 
        user.loc[i, ['city', 'region', 'country']] = [not_know_li[0],not_know_li[0],not_know_li[1]]

In [26]:
incompleti = user.loc[three,]

In [27]:
for i in respect['clean_location']:
    indice = incompleti.loc[incompleti['clean_location'].str.contains(i)].index
    if len(indice) > 0:
        city_i, region_i, country_i = i.split(',')[0],i.split(',')[1],i.split(',')[2]
        user.loc[indice, ['city','region', 'country']] = [city_i, region_i, country_i]

In [28]:
user = pd.DataFrame(user)

Se le precedenti condizioni non sono verificate, facciamo un ulteriore controllo in cui teniamo solo gli utlimi 3 valori e poi verifichiamo che tutti i campi 'city','region', 'country', quando 'clean_location' è presente (non Nan), siano stati creati correttamente:

In [29]:
for i in user.loc[user['city']== '',].index:
    li = user.loc[i,'clean_location'].split(',')
    if li != ['']:
        user.loc[i, ['city', 'region', 'country']] = [li[-3],li[-2],li[-1]]

for i in user.loc[user['region']== '',].index:
    li = user.loc[i,'clean_location'].split(',')
    if li != ['']:
        user.loc[i, ['city', 'region', 'country']] = [li[-3],li[-2],li[-1]]

for i in user.loc[user['country']== '',].index:
    li = user.loc[i,'clean_location'].split(',')
    if li != ['']:
        user.loc[i, ['city', 'region', 'country']] = [li[-3],li[-2],li[-1]]

In [30]:
pd.unique(user.loc[user['city']== '','clean_location'])

array([''], dtype=object)

In [31]:
pd.unique(user.loc[user['region']== '','clean_location'])

array([''], dtype=object)

In [32]:
pd.unique(user.loc[user['country']== '','clean_location'])

array([''], dtype=object)

In [33]:
user

,User-ID,Location,Age,clean_location,city,region,country
0,1,"nyc, new york, usa",NaN,"nyc, new york, usa",nyc,new york,usa
1,2,"stockton, california, usa",18.0,"stockton, california, usa",stockton,california,usa
2,3,"moscow, yukon territory, russia",NaN,"moscow, yukon territory, russia",moscow,yukon territory,russia
3,4,"porto, v.n.gaia, portugal",17.0,"porto, vngaia, portugal",porto,vngaia,portugal
4,5,"farnborough, hants, united kingdom",NaN,"farnborough, hants, united kingdom",farnborough,hants,united kingdom
...,...,...,...,...,...,...,...
278853,278854,"portland, oregon, usa",NaN,"portland, oregon, usa",portland,oregon,usa
278854,278855,"tacoma, washington, united kingdom",50.0,"tacoma, washington, united kingdom",tacoma,washington,united kingdom
278855,278856,"brampton, ontario, canada",NaN,"brampton, ontario, canada",brampton,ontario,canada
278856,278857,"knoxville, tennessee, usa",NaN,"knoxville, tennessee, usa",knoxville,tennessee,usa


## RICHIESTA 2:  For each book in the Books dataset, compute its average rating

In [ ]:
book.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


In [ ]:
avg = book_rat[book_rat['ISBN'].isin(book['ISBN'])].groupby('ISBN',as_index = False)['Book-Rating'].mean()

In [ ]:
avg.head(10)

,ISBN,Book-Rating
0,0000913154,8.0
1,0001010565,0.0
2,0001046438,9.0
3,0001046713,0.0
4,000104687X,6.0
5,0001046934,0.0
6,0001047213,9.0
7,0001047647,0.0
8,0001047663,0.0
9,0001047868,0.0


## RICHIESTA 3:   For each book in the GoodBooks dataset, compute its average rating.

In [ ]:
good_book.head(3)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...


In [ ]:
good_book.columns

Index(['book_id', 'goodreads_book_id', 'best_book_id', 'work_id',
       'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year',
       'original_title', 'title', 'language_code', 'average_rating',
       'ratings_count', 'work_ratings_count', 'work_text_reviews_count',
       'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5',
       'image_url', 'small_image_url'],
      dtype='object')

In [80]:
good_book = pd.DataFrame(good_book)

In [82]:
our_avg = round((good_book['ratings_1']+good_book['ratings_2']*2 +good_book['ratings_3']*3+good_book['ratings_4']*4 +good_book['ratings_5']*5)/good_book['work_ratings_count'],2)
our_avg

0     4.34
1     4.44
2     3.57
3     4.25
4     3.89
      ... 
94    4.06
95    3.88
96    3.98
97    4.22
98    3.87
Length: 99, dtype: float64

Notiamo che coincide con la colonna già pre esistente 'average_rating'

In [ ]:
good_book['average_rating']

0     4.34
1     4.44
2     3.57
3     4.25
4     3.89
      ... 
94    4.06
95    3.88
96    3.98
97    4.22
98    3.87
Name: average_rating, Length: 99, dtype: float64

In [83]:
sum(our_avg-good_book['average_rating'])

0.0

## RICHIESTA 4:   Merge together all rows sharing the same book title, author and publisher. We will call the resulting dataset merged books. The books that have not been merged together will not appear in merged books

Inserendo all'interno di pd.duplicated 'keep =False', viene associato il valore True a tutte le osservazioni che si ripetono

In [35]:
merged_book = book.loc[book.duplicated(subset=['Book-Title', 'Book-Author','Publisher'], keep=False),]
merged_book.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
14,1558746218,A Second Chicken Soup for the Woman's Soul (Ch...,Jack Canfield,1998,Health Communications,http://images.amazon.com/images/P/1558746218.0...,http://images.amazon.com/images/P/1558746218.0...,http://images.amazon.com/images/P/1558746218.0...
29,0684823802,OUT OF THE SILENT PLANET,C.S. Lewis,1996,Scribner,http://images.amazon.com/images/P/0684823802.0...,http://images.amazon.com/images/P/0684823802.0...,http://images.amazon.com/images/P/0684823802.0...
41,055321215X,Pride and Prejudice,Jane Austen,1983,Bantam,http://images.amazon.com/images/P/055321215X.0...,http://images.amazon.com/images/P/055321215X.0...,http://images.amazon.com/images/P/055321215X.0...


## RICHIESTA 5: For each book in merged books compute its average rating 
The average is computed considering all books in books that have been merged. 


In [36]:
avg_rat_merge = pd.merge(merged_book, book_rat, on = 'ISBN', how = 'left')
avg_rat_merge.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,1558746218,A Second Chicken Soup for the Woman's Soul (Ch...,Jack Canfield,1998,Health Communications,http://images.amazon.com/images/P/1558746218.0...,http://images.amazon.com/images/P/1558746218.0...,http://images.amazon.com/images/P/1558746218.0...,8.0,0.0
1,1558746218,A Second Chicken Soup for the Woman's Soul (Ch...,Jack Canfield,1998,Health Communications,http://images.amazon.com/images/P/1558746218.0...,http://images.amazon.com/images/P/1558746218.0...,http://images.amazon.com/images/P/1558746218.0...,3363.0,0.0
2,1558746218,A Second Chicken Soup for the Woman's Soul (Ch...,Jack Canfield,1998,Health Communications,http://images.amazon.com/images/P/1558746218.0...,http://images.amazon.com/images/P/1558746218.0...,http://images.amazon.com/images/P/1558746218.0...,61501.0,7.0


In [37]:
avg_rat_merge.groupby(['Book-Title', 'Book-Author','Publisher'],as_index = False)['Book-Rating'].mean()

,Book-Title,Book-Author,Publisher,Book-Rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,3.000000
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,1.888889
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,7.000000
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3.833333
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,0.000000
...,...,...,...,...
4720,Zia,Scott O'Dell,Laurel-Leaf Books,2.615385
4721,Zia,Scott O'Dell,Yearling Books,1.636364
4722,Zimmermann Telegram,Barbara Tuchman,Ballantine Books,2.000000
4723,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.000000


## RICHIESTA 6: For each book in merged books compute the minimum and maximum of the average ratings over all corresponding books in the books dataset.
Hence for each book in merged books we will have exactly two values (a minimum and a maximum)

In [ ]:
avg_rat_merge.groupby(['ISBN'],as_index = False)['Book-Rating'].agg(['min','max'])

,min,max
ISBN,,
0001841572,0.0,5.0
0001857258,10.0,10.0
0002200813,0.0,0.0
0002200821,0.0,0.0
0002228394,7.0,9.0
...,...,...
B00016560C,0.0,0.0
B00016560M,0.0,0.0
B0001FZGBC,7.0,7.0


## RICHIESTA 7: For each book in goodbooks, compute the list of its authors. Assuming that the number of reviews with a text (column work_text_reviews_count) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the shared number of reviews with a text

In [38]:
review_author = good_book[['isbn','authors','work_text_reviews_count']]
review_author = pd.DataFrame(review_author)

In [39]:
review_author.head(3)

,isbn,authors,work_text_reviews_count
0,439023483,Suzanne Collins,155254
1,439554934,"J.K. Rowling, Mary GrandPrÃ©",75867
2,316015849,Stephenie Meyer,95009


Creiamo la lista che contiene gli autori con pd.Series.str.split(',') in modo da poter calcolare la lunghezza con .str.len e sfruttare il comando .explode

In [41]:
review_author['authors_li'] = review_author.authors.str.split(',')
review_author.head(3)

,isbn,authors,work_text_reviews_count,authors_li
0,439023483,Suzanne Collins,155254,[Suzanne Collins]
1,439554934,"J.K. Rowling, Mary GrandPrÃ©",75867,"[J.K. Rowling, Mary GrandPrÃ©]"
2,316015849,Stephenie Meyer,95009,[Stephenie Meyer]


In [42]:
review_author['num_authors'] = review_author['authors_li'].str.len()
review_author.head(3)

,isbn,authors,work_text_reviews_count,authors_li,num_authors
0,439023483,Suzanne Collins,155254,[Suzanne Collins],1
1,439554934,"J.K. Rowling, Mary GrandPrÃ©",75867,"[J.K. Rowling, Mary GrandPrÃ©]",2
2,316015849,Stephenie Meyer,95009,[Stephenie Meyer],1


In [43]:
review_author['shared_num_reviews'] = review_author['work_text_reviews_count']/review_author['num_authors']
review_author.head(10)

,isbn,authors,work_text_reviews_count,authors_li,num_authors,shared_num_reviews
0,439023483,Suzanne Collins,155254,[Suzanne Collins],1,155254.0
1,439554934,"J.K. Rowling, Mary GrandPrÃ©",75867,"[J.K. Rowling, Mary GrandPrÃ©]",2,37933.5
2,316015849,Stephenie Meyer,95009,[Stephenie Meyer],1,95009.0
3,61120081,Harper Lee,72586,[Harper Lee],1,72586.0
4,743273567,F. Scott Fitzgerald,51992,[F. Scott Fitzgerald],1,51992.0
5,525478817,John Green,140739,[John Green],1,140739.0
6,618260307,J.R.R. Tolkien,37653,[J.R.R. Tolkien],1,37653.0
7,316769177,J.D. Salinger,44920,[J.D. Salinger],1,44920.0
8,1416524797,Dan Brown,25112,[Dan Brown],1,25112.0
9,679783261,Jane Austen,49152,[Jane Austen],1,49152.0


Il comando explode applicato alla lista di autori, spacca la lista duplicando le linee e associando i rispettivi valori:

In [44]:
review_author.explode('authors_li').groupby('authors_li', as_index = False)['shared_num_reviews'].sum()

,authors_li,shared_num_reviews
0,Alan R. Clarke,27890.500000
1,B.M. Mooyaart-Doubleday,6941.666667
2,Bernard Knox,1620.200000
3,CelÃ¢l Ãster,15172.666667
4,David J. Skal,5754.333333
...,...,...
105,Veronica Roth,156896.000000
106,William Golding,26886.000000
107,William Goldman,15630.000000
108,William Shakespeare,7389.000000


## RICHIESTA 8: For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.



In [45]:
review_author_year = pd.merge(review_author, good_book[['isbn','original_publication_year']], on = 'isbn', how = 'left' )

In [48]:
review_author_year = review_author_year.explode('authors_li')
review_author_year.head(3)

,isbn,authors,work_text_reviews_count,authors_li,num_authors,shared_num_reviews,original_publication_year
0,439023483,Suzanne Collins,155254,Suzanne Collins,1,155254.0,2008.0
1,439554934,"J.K. Rowling, Mary GrandPrÃ©",75867,J.K. Rowling,2,37933.5,1997.0
1,439554934,"J.K. Rowling, Mary GrandPrÃ©",75867,Mary GrandPrÃ©,2,37933.5,1997.0


In [49]:
author_by_year = review_author_year.groupby(['authors_li','original_publication_year'], as_index = False)['shared_num_reviews'].sum()

In [50]:
author_by_year = author_by_year.loc[author_by_year.groupby('original_publication_year')['shared_num_reviews'].idxmax()]
author_by_year.head(15)

,authors_li,original_publication_year,shared_num_reviews
2,Bernard Knox,-720.0,1620.200000
35,Robert Jackson,1595.0,7389.000000
37,Ros Ballaster,1811.0,3842.000000
85,Jane Austen,1813.0,49152.000000
25,Maurice Hindle,1818.0,6664.333333
27,Michael Mason,1847.0,15606.000000
13,Hablot Knight Browne,1859.0,4364.333333
101,Louisa May Alcott,1868.0,17090.000000
12,Guy Cardwell,1884.0,4149.333333
14,Jeffrey Eugenides,1891.0,9823.500000


## RICHIESTA 9: Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

In [51]:
good_book = good_book.rename(columns = {'isbn': 'ISBN'})

In [52]:
both = pd.merge(book, good_book, on= 'ISBN', how = 'inner')
both.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,book_id,goodreads_book_id,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1400032717,The Curious Incident of the Dog in the Night-T...,Mark Haddon,2004,Vintage,http://images.amazon.com/images/P/1400032717.0...,http://images.amazon.com/images/P/1400032717.0...,http://images.amazon.com/images/P/1400032717.0...,60,1618,...,867553,915790,35796,21049,58117,218218,355925,262481,https://images.gr-assets.com/books/1479863624m...,https://images.gr-assets.com/books/1479863624s...
1,014028009X,Bridget Jones's Diary,Helen Fielding,1999,Penguin Books,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,75,227443,...,736498,754498,8157,28973,59309,198555,254836,212825,https://images.gr-assets.com/books/1292060045m...,https://images.gr-assets.com/books/1292060045s...
2,014038572X,The Outsiders (Now in Speak!),S. E. Hinton,1997,Puffin Books,http://images.amazon.com/images/P/014038572X.0...,http://images.amazon.com/images/P/014038572X.0...,http://images.amazon.com/images/P/014038572X.0...,90,231804,...,659248,680437,22662,10178,34346,134244,227250,274419,https://images.gr-assets.com/books/1442129426m...,https://images.gr-assets.com/books/1442129426s...


In [85]:
rating1 = book_rat.groupby('ISBN', as_index=False)['Book-Rating'].mean()
rating1.head(3)

,ISBN,Book-Rating
0,0330299891,3.0
1,0375404120,1.5
2,0586045007,0.0


In [84]:
unione_ulti = pd.merge(both, rating1, on='ISBN', how='left')
unione_ulti.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,book_id,goodreads_book_id,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,Book-Rating
0,1400032717,The Curious Incident of the Dog in the Night-T...,Mark Haddon,2004,Vintage,http://images.amazon.com/images/P/1400032717.0...,http://images.amazon.com/images/P/1400032717.0...,http://images.amazon.com/images/P/1400032717.0...,60,1618,...,915790,35796,21049,58117,218218,355925,262481,https://images.gr-assets.com/books/1479863624m...,https://images.gr-assets.com/books/1479863624s...,4.813187
1,014028009X,Bridget Jones's Diary,Helen Fielding,1999,Penguin Books,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,75,227443,...,754498,8157,28973,59309,198555,254836,212825,https://images.gr-assets.com/books/1292060045m...,https://images.gr-assets.com/books/1292060045s...,3.751852
2,014038572X,The Outsiders (Now in Speak!),S. E. Hinton,1997,Puffin Books,http://images.amazon.com/images/P/014038572X.0...,http://images.amazon.com/images/P/014038572X.0...,http://images.amazon.com/images/P/014038572X.0...,90,231804,...,680437,22662,10178,34346,134244,227250,274419,https://images.gr-assets.com/books/1442129426m...,https://images.gr-assets.com/books/1442129426s...,4.460674


In [56]:
unione_ulti['Diff_avg_rat'] = unione_ulti['Book-Rating'] - unione_ulti['average_rating'] 
unione_ulti.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,book_id,goodreads_book_id,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,Book-Rating,Diff_avg_rat
0,1400032717,The Curious Incident of the Dog in the Night-T...,Mark Haddon,2004,Vintage,http://images.amazon.com/images/P/1400032717.0...,http://images.amazon.com/images/P/1400032717.0...,http://images.amazon.com/images/P/1400032717.0...,60,1618,...,35796,21049,58117,218218,355925,262481,https://images.gr-assets.com/books/1479863624m...,https://images.gr-assets.com/books/1479863624s...,4.813187,0.963187
1,014028009X,Bridget Jones's Diary,Helen Fielding,1999,Penguin Books,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,75,227443,...,8157,28973,59309,198555,254836,212825,https://images.gr-assets.com/books/1292060045m...,https://images.gr-assets.com/books/1292060045s...,3.751852,0.001852
2,014038572X,The Outsiders (Now in Speak!),S. E. Hinton,1997,Puffin Books,http://images.amazon.com/images/P/014038572X.0...,http://images.amazon.com/images/P/014038572X.0...,http://images.amazon.com/images/P/014038572X.0...,90,231804,...,22662,10178,34346,134244,227250,274419,https://images.gr-assets.com/books/1442129426m...,https://images.gr-assets.com/books/1442129426s...,4.460674,0.400674


## RICHIESTA 10:  Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

In [86]:
user.head(3)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN


In [59]:
print('Minima età: ',np.nanmin(user['Age']))
print('Massima età: ',np.nanmax(user['Age']))

Minima età:  0.0
Massima età:  244.0


Creiamo una funzione per fare lo split del dataset:

In [60]:
def subset(year, data = user):
    if year != 'missing':
        new_data = data[data['Age'].isin(year)] 
    else:
        new_data = data[data['Age'].isna()] 
    return new_data

Creiamo la lista degli anni:

In [62]:
lista_year = ['missing']+[0]+list(range(15,244+10, 10))
lista_year[0:10]

['missing', 0, 15, 25, 35, 45, 55, 65, 75, 85]

I vari pandas dataframe saranno contenuti in un dizionario:

In [63]:
diz_pd = {}
for i in range(0, len(lista_year)-1):
    if i != 0:
        year = list(range(lista_year[i],lista_year[i+1]))
        name = str(year[0])+ '_' +str(year[-1])
        diz_pd[name] = subset(year)
    else:
        diz_pd['missing'] = subset('missing')

Per accedere al dataset di interesse si una la chiave composta da:  'età inferiore + _ + età superiore'

gli estremi sono inclusi

In [65]:
diz_pd['0_14'].head(3)

,User-ID,Location,Age,clean_location,city,region,country
10,11,"melbourne, victoria, australia",14.0,"melbourne, victoria, australia",melbourne,victoria,australia
18,19,"weston, ,",14.0,weston,weston,weston,weston
40,41,"santee, california, usa",14.0,"santee, california, usa",santee,california,usa


In [66]:
diz_pd['missing'].head(3)

,User-ID,Location,Age,clean_location,city,region,country
0,1,"nyc, new york, usa",NaN,"nyc, new york, usa",nyc,new york,usa
2,3,"moscow, yukon territory, russia",NaN,"moscow, yukon territory, russia",moscow,yukon territory,russia
4,5,"farnborough, hants, united kingdom",NaN,"farnborough, hants, united kingdom",farnborough,hants,united kingdom


## RICHIESTA 11:  Find the books that appear only in the goodbooks datasets

In [67]:
good_book[-good_book['ISBN'].isin(book['ISBN'])].head(3)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,ISBN,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...


In [68]:
print('Book contiene ',good_book.shape[0] - good_book[-good_book['ISBN'].isin(book['ISBN'])].shape[0], ' libri di goodbook')

Book contiene  5  libri di goodbook


## RICHIESTA 12: Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [ ]:
book.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


In [69]:
pair = book[['Book-Author', 'Book-Title', 'ISBN']]

In [70]:
pair = pair.groupby(['Book-Author', 'Book-Title'], as_index = False)['ISBN'].count()

In [71]:
pair.loc[pair['ISBN'] == pair['ISBN'].max(),]

,Book-Author,Book-Title,ISBN
150649,Louisa May Alcott,Little Women,21


## RICHIESTA 13: Find the author with the highest average rating according to the goodbooks datasets

In [73]:
good_book.head(3)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,ISBN,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...


In [74]:
good_book['authors_li'] = good_book.authors.str.split(',')

In [75]:
good_book = good_book.explode('authors_li').groupby('authors_li', as_index = False)['average_rating'].mean().sort_values('average_rating', ascending = False)

In [76]:
good_book.head(3)

,authors_li,average_rating
32,Rufus Beck,4.530000
18,Mary GrandPrÃ©,4.497143
64,J.K. Rowling,4.497143


In [77]:
good_book.loc[good_book['average_rating'] == good_book['average_rating'].max(),]

,authors_li,average_rating
32,Rufus Beck,4.53
